---
---
# NEURAL STYLE TRANSFER
---
---

# STEP 1 - Generating The Content
* E.g. given an image, can we recrate the same image

In [1]:
from __future__ import print_function, division
from builtins import range, input

from keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.models import Models, Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image

import keras.backend as K #we can't use fit function, we need this to reach gradients to use gradient descent or another loss
import numpy as np
from scipy.optimize import fmin_l_bfgs_b #key algorithm

ModuleNotFoundError: No module named 'keras.layers.convolutional'

In [ ]:
def VGG16_AvgPool(shape):
    #we want to account for features across the entire image
    #so we get rid of the maxpool which throws away information
    vgg = VGG16(input_shape=shape,
                weights='imagenet',
                include_top=False)
    new_model = Sequential()
    for layer in vgg.layers:
        if layer.__class__ == MaxPooling2D:
            #replace it with average pooling
            new_model.add(AveragePooling2D())
        else:
            new_model.add(layer)
    return new_model